In [1]:
import pickle
import random

import numpy as np

matches_df = pickle.load(open("database/matches_df.pck", "rb"))

In [2]:
winner_list = []
market_val_diff = []
goals_diff = []
for _, row in matches_df.iterrows():
    if row["Team Home ID"] == row["Winner Team ID"]:
        winner_list.append("Home")
    elif row["Team Away ID"] == row["Winner Team ID"]:
        winner_list.append("Away")
    elif row["Winner Team ID"] == 0:
        winner_list.append("Draw")
    market_val_diff.append(row["Market Value Home"] - row["Market Value Away"])
    goals_diff.append(row["Goals Home"] - row["Goals Away"])
matches_df["Result"] = winner_list
matches_df["Market Value Difference"] = market_val_diff
matches_df["Goals Difference"] = goals_diff

In [3]:
import asyncio
import nest_asyncio
from database import dbconn
import importlib

importlib.reload(dbconn)

nest_asyncio.apply()

loop = asyncio.get_event_loop()



In [10]:
import pandas as pd

dict_team_data = dict()

for team_id in set(matches_df["Team Home ID"]):
    team_data_matches_home, team_data_matches_away = loop.run_until_complete(dbconn.get_matches_by_team(team_id))
    team_data_matches = [[x.id, x.goalsHome, x.goalsAway, x.winnerTeamId] for x in team_data_matches_home] + [[x.id, x.goalsHome, x.goalsAway, x.winnerTeamId] for x in team_data_matches_away]
    team_data_matches = sorted(team_data_matches, key=lambda x: x[0])
    for i in range(len(team_data_matches)):
        if team_data_matches[i][3] == team_id:
            team_data_matches[i][3] = "Won"
        elif team_data_matches[i][3] == None:
            team_data_matches[i][3] = "Draw"
        else:
            team_data_matches[i][3] = "Lost"
    dict_team_data[team_id] = pd.DataFrame(team_data_matches)

In [22]:
matches_form = []
for _, row in matches_df.iterrows():
    home_form = dict_team_data[row["Team Home ID"]]
    home_form = home_form[home_form[0] < row["Match ID"]].iloc[-5:]
    home_form_5 = list(home_form[3])
    for i in range(5 - len(home_form_5)):
        home_form_5 = ["Draw"] + home_form_5
    away_form = dict_team_data[row["Team Away ID"]]
    away_form = away_form[away_form[0] < row["Match ID"]].iloc[-5:]
    away_form_5 = list(away_form[3])
    for i in range(5 - len(away_form_5)):
        away_form_5 = ["Draw"] + away_form_5
    matches_form.append(home_form_5 + away_form_5)

In [37]:
pd.concat([matches_df, pd.DataFrame(matches_form, columns=["Home_Pre" + str(x) for x in range(5, 0, -1)] + ["Away_Pre" + str(x) for x in range(5, 0, -1)])], axis=1)

,Match ID,Team Home ID,Team Away ID,Market Value Home,Market Value Away,Goals Home,Goals Away,Winner Team ID,Result,Market Value Difference,...,Home_Pre5,Home_Pre4,Home_Pre3,Home_Pre2,Home_Pre1,Away_Pre5,Away_Pre4,Away_Pre3,Away_Pre2,Away_Pre1
0,19702,7,134,301.25,72.00,2,1,7,Home,229.25,...,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw
1,19703,87,175,82.33,70.95,2,1,87,Home,11.38,...,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw
2,19704,16,131,88.62,103.75,0,1,131,Away,-15.13,...,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw
3,19705,55,9,76.65,178.35,2,2,0,Draw,-101.70,...,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw
4,19706,112,81,63.10,47.23,1,1,0,Draw,15.87,...,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw,Draw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,61146,87,175,234.00,207.72,5,1,87,Home,26.28,...,Won,Lost,Draw,Won,Draw,Lost,Draw,Draw,Lost,Lost
3056,61147,81,91,125.53,224.75,2,2,0,Draw,-99.22,...,Lost,Draw,Lost,Won,Won,Lost,Lost,Draw,Lost,Draw
3057,61148,16,65,168.12,92.88,2,1,16,Home,75.24,...,Lost,Draw,Lost,Draw,Draw,Won,Won,Won,Won,Lost
3058,61149,80,129,110.70,49.55,3,2,80,Home,61.15,...,Won,Won,Won,Draw,Won,Draw,Lost,Lost,Won,Won


['Home_Pre5',
 'Home_Pre4',
 'Home_Pre3',
 'Home_Pre2',
 'Home_Pre1',
 'Away_Pre5',
 'Away_Pre4',
 'Away_Pre3',
 'Away_Pre2',
 'Away_Pre1']